In [ ]:
import corner
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tftables
import time
from tqdm import tqdm_notebook as tqdm

import data_loader
import model_short as model
import toy_data_loader

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True, precision=4)

In [ ]:
class hps:
    pass
hps.n_levels = 3 # number of splits
hps.depth = 3 # number of layers in revnet
hps.width = 16 # channels in revnet layers
hps.polyak_epochs = 1
hps.beta1 = .9 # learning rate annealing factor
hps.weight_decay = 1 # learning rate annealing factor
hps.lr = .001 # base learning rate
hps.n_data = 16000 # number of input spectra
hps.batch_size = 50 # number of spectra in a batch
hps.n_batches = int(hps.n_data / hps.batch_size)
hps.n_bins = 2**12

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
# select real or toy data by uncommenting the appropriate line
# real data must have n_data=8000, n_bins=40000
#input_stream, initialize_input_stream, data_init = data_loader.create_data_loader(
input_stream, initialize_input_stream, data_init = toy_data_loader.create_data_loader(
    sess, hps.batch_size, hps.n_data, hps.n_bins
)
'''
spectra = np.load('sample_short.npz')['spectra']
sqrt = np.sqrt(spectra)

# add noise
#sums = spectra.sum(axis=1)
#sqrtsums = sqrt.sum(axis=1)
#As = .02 * sums / (np.sqrt(2 / 3.14) * sqrtsums)
#noise = np.random.normal(scale=(np.repeat(As[:, np.newaxis], hps.n_bins, axis=1) * sqrt))
#print((np.abs(noise).sum(axis=1) / spectra.sum(axis=1)))

scaled_spectra = spectra / spectra.std(axis=1)[:, np.newaxis]
#scaled_spectra = (spectra + noise) / (spectra + noise).std(axis=1)[:, np.newaxis]
centered_spectra = scaled_spectra - scaled_spectra.mean(axis=1)[:, np.newaxis]
#normalized_spectra = spectra / np.max(spectra, axis=1)[:, np.newaxis]

def create_data_loader(sess, data, batch_size):
    placeholder_data = tf.compat.v1.placeholder(tf.float32, data.shape)
    dataset = tf.data.Dataset.from_tensor_slices(placeholder_data)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    input_stream = iterator.get_next()
    
    def initialize_input_stream():
        sess.run(iterator.initializer, feed_dict={placeholder_data: data})
    
    initialize_input_stream()
    data_init = sess.run(input_stream)
    return input_stream, initialize_input_stream, data_init

input_stream, initialize_input_stream, data_init = create_data_loader(
    sess, centered_spectra[:, :, np.newaxis], hps.batch_size
)'''

In [ ]:
print(data_init.shape)
plt.figure(figsize=(15, 5))
for spectrum in data_init[:25]:
    plt.plot(spectrum)

In [ ]:
with tf.device("/device:GPU:0"):
    m = model.model(sess, hps, input_stream, data_init)

In [ ]:
m.restore('models/model-200318-095826')

In [ ]:
i = np.random.randint(0, hps.batch_size)
spectrum = data_init[i:i+1, :, :]
latent_rep = m.encode(spectrum)
reconstruction = m.decode(latent_rep)
print(i)

In [ ]:
window = (1850, 2200) #(12000, 14000)
window_size = (window[1] - window[0])
window_fraction = window_size / hps.n_bins

In [ ]:
plt.figure(figsize=(10, 7))

plt.subplot(2, 1, 1)
plt.plot(np.squeeze(reconstruction))
plt.plot(np.squeeze(spectrum))
plt.axvline(window[0])
plt.axvline(window[1])

plt.subplot(2, 1, 2)
plt.plot(range(*window), np.squeeze(reconstruction)[window[0]:window[1]])
plt.plot(range(*window), np.squeeze(spectrum)[window[0]:window[1]])

In [ ]:
def create_gaussian_kernel(size, mean, std):
    d = tf.distributions.Normal(tf.cast(mean, tf.float32), tf.cast(std, tf.float32))
    vals = d.prob(tf.range(start=-int(size/2), limit=int(size/2)+1, dtype=tf.float32))

    kernel = vals[:, np.newaxis, np.newaxis]
    return kernel / tf.reduce_sum(kernel)

In [ ]:
gaussian_kernel = create_gaussian_kernel(51, 0, 25)
derivative_kernel = tf.constant([[[-hps.n_bins / 2]], [[0]], [[hps.n_bins / 2]]])

In [ ]:
smoothed = tf.nn.conv1d(m.decoded_spectra, gaussian_kernel, padding="SAME")
first_derivative = tf.nn.conv1d(smoothed, derivative_kernel, padding="SAME")
smoothed_first_derivative = tf.nn.conv1d(first_derivative, gaussian_kernel, padding="SAME")
second_derivative = tf.nn.conv1d(smoothed_first_derivative, derivative_kernel, padding="SAME")

In [ ]:
smoothed_spectra = sess.run(smoothed, {m.z_placeholder: latent_rep})
first_derivative_spectra = sess.run(first_derivative, {m.z_placeholder: latent_rep})
second_derivative_spectra = sess.run(second_derivative, {m.z_placeholder: latent_rep})

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(np.squeeze(reconstruction))
plt.plot(np.squeeze(smoothed_spectra))
plt.plot(np.squeeze(first_derivative_spectra) / first_derivative_spectra.std())
plt.plot(np.squeeze(second_derivative_spectra) / second_derivative_spectra.std())
plt.ylim(-5, 5)

In [ ]:
# outside window
left_squared_error = tf.reduce_sum((spectrum[:, :window[0]] - m.decoded_spectra[:, :window[0]])**2)
right_squared_error = tf.reduce_sum((spectrum[:, window[1]:] - m.decoded_spectra[:, window[1]:])**2)
outside_cost = left_squared_error + right_squared_error

# inside window
#inside_cost = tf.reduce_sum((spectrum[:, window[0]:window[1]] - m.decoded_spectra[:, window[0]:window[1]])**2)
inside_cost = -tf.reduce_sum(second_derivative[:, window[0]:window[1]]**2)

# likelihood
logp = -.5 * np.sum(m.z_placeholder**2)
beta = 1e11

cost = inside_cost - outside_cost + beta * logp # maximize inside cost and likelihood. minimize outside cost
gradient = tf.gradients(cost, m.z_placeholder)

In [ ]:
grads = []
latent_reps = [latent_rep]

In [ ]:
for _ in range(1000):
    grads.append(sess.run(gradient, {m.z_placeholder: latent_reps[-1]})[0])
    step_size = .01 / np.linalg.norm(grads[-1][0])
    latent_reps.append(latent_reps[-1] + step_size * grads[-1][0])

In [ ]:
# exploration analysis
latent_reps_np = np.array(latent_reps)
grads_np = np.array(grads)
print_freq = int(len(grads) / 20) # when plotting changes over time, plot around 20 things

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(3, 2, 1)
for i in [0, 1, 2, 3]:
    plt.plot(latent_reps_np.mean(axis=(0, 1))[:, i])
plt.xlabel('component position')
plt.ylabel('latent rep (avg over steps)')

plt.subplot(3, 2, 2)
for i in [0, 1, 2, 3]:
    plt.plot(grads_np.mean(axis=(0, 1))[:, i])
plt.xlabel('component position')
plt.ylabel('gradient (avg over steps)')

plt.subplot(3, 2, 3)
plt.plot([np.linalg.norm(l) for l in latent_reps])
plt.xlabel('step')
plt.ylabel('norm of latent representation')

plt.subplot(3, 2, 4)
plt.plot([np.linalg.norm(g) for g in grads])
plt.xlabel('step')
plt.ylabel('norm of gradient')

plt.subplot(3, 2, 5)
for i in range(0, len(latent_reps), print_freq):
    plt.plot(latent_reps_np[i].sum(axis=(0, 2)))
plt.xlim(100, 150)
plt.xlabel('component position')
plt.ylabel('latent rep (sum over channels) over time')

plt.subplot(3, 2, 6)
for i in range(0, len(grads), print_freq):
    plt.plot(grads_np[i].sum(axis=(0, 2)))
plt.xlim(100, 150)
plt.xlabel('component position')
plt.ylabel('gradient (sum over channels) over time')

plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
plt.plot(np.squeeze(reconstruction))
for i in range(0, len(latent_reps), print_freq):
    plt.plot(np.squeeze(m.decode(latent_reps[i])))
plt.axvline(window[0])
plt.axvline(window[1])

plt.subplot(2, 1, 2)
plt.plot(range(*window), np.squeeze(reconstruction)[window[0]:window[1]])
for i in range(0, len(latent_reps), print_freq):
    plt.plot(range(*window), np.squeeze(m.decode(latent_reps[i]))[window[0]:window[1]])